# INTRO

In [ ]:
from google.colab import drive
import os

import pandas as pd
import numpy as np
import re

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

# IMPORT CSV _ lista de doutores

In [ ]:
df = pd.read_csv('doutoresFeaturesParte_1.csv', encoding='UTF-8', nrows=39000)

df.dropna(subset=["GrandesAreas"], inplace=True)
df.dropna(subset=["Genero"], inplace=True)
df.dropna(subset=["ExclusividadeDoutorado"], inplace=True)


df = df.fillna(' ')

df = df.astype({
    'IDLattes': int,
    'DuracaoDoutorado': float,
    'Genero': str,
    'GrandesAreas': str,
    'ExclusividadeDoutorado': str,
    'Publicacoes': float,
    'IdadeAcademica': str,
    'ListaFiliacoes': str,
    'UltimaFiliacao': str,
})

df_doutores = df.copy(deep=True)
df_doutores

# Nova Coluna com Resultado
df_doutores['HistoricoFiliacoesEmIESInternacional'] = ''
df_doutores['UltimaFiliacaoEmIES'] = ''

In [ ]:
df_doutores

In [ ]:
df_doutores['GrandesAreas'].unique()

# IMPORT CSV _ IES Brasil

In [ ]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni_br = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_br = df_uni_br[['NOME_DA_IES', 'SIGLA']]
df_uni_br = df_uni_br.fillna('')
df_uni_br = df_uni_br.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_br

# IMPORT CSV _ IES Mundial

In [ ]:
# Lista de IES Mundial
# https://zenodo.org/records/13965926
# https://zenodo.org/records/13965926

df_uni_world = pd.read_csv('ror.csv', encoding='UTF-8')

df_uni_world = df_uni_world[['name', 'aliases', 'acronyms', 'country.country_code']]
df_uni_world = df_uni_world.rename(columns={
    'country.country_code': 'PAIS',
    'name': 'NOME_DA_IES',
    'acronyms': 'SIGLA',
    'aliases': 'ALIASES'
})

# IES BR foram removidos pois os nomes não estão em inglês e assim ficaria duplicado em relação a dataset do MEC
df_uni_world = df_uni_world[df_uni_world['PAIS'] != 'BR']

df_uni_world = df_uni_world.fillna('')

df_uni_world

# PRE-PROCESSAMENTO

In [ ]:
def clean_names(df, column_name, remove_semicolon=True):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace e ;
    if remove_semicolon:
        df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)
    else:
        df[column_name] = df[column_name].str.replace('[^\w\s;]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [ ]:
df_doutores = df_doutores.apply(lambda x:x.astype(str).str.upper())
df_doutores = clean_names(df_doutores, 'ListaFiliacoes', False)
df_doutores = clean_names(df_doutores, 'UltimaFiliacao', False)

df_uni_br = df_uni_br.apply(lambda x:x.astype(str).str.upper())
df_uni_br = clean_names(df_uni_br, 'NOME_DA_IES')

In [ ]:
# Split lista de IES por inicio do nome
df_uni_br.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_br.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

# Merge IES df

In [ ]:
df_uni_br['PAIS'] = 'BR'
df_uni_br['ALIASES'] = ''

print( (df_uni_world.shape[0] + df_uni_br.shape[0]), ' = ', df_uni_world.shape[0], ' + ', df_uni_br.shape[0])

df_uni = pd.concat([df_uni_br, df_uni_world])

df_uni["SIGLA"] = df_uni["SIGLA"].apply(lambda x: x.split(";")[0].strip())
clean_names(df_uni, "SIGLA")

In [ ]:
df_uni_sigla = df_uni.copy(deep=True)

rows_to_drop = df_uni_sigla[df_uni_sigla['SIGLA']==''].index
df_uni_sigla.drop(rows_to_drop, inplace=True)

In [ ]:
df_uni_sigla

# PLN

## TF_IDF

In [ ]:
def get_uni_list_df(nome_instituicao):
    if 'INSTITUTO' in nome_instituicao:
        return df_uni_instituto, tfidf_matrix_instituto
    elif 'FACULDADE' in nome_instituicao:
        return df_uni_faculdades, tfidf_matrix_faculdades
    elif 'ESCOLA' in nome_instituicao:
        return df_uni_escola, tfidf_matrix_escola
    elif 'UNIVERSIDADE' in nome_instituicao:
        return df_uni_universidade, tfidf_matrix_universidade
    elif 'INSTITUICAO' in nome_instituicao:
        return df_uni_instituicao, tfidf_matrix_instituicao

    return df_uni_outros, tfidf_matrix_outros

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF : Lista IES Brasil
vectorizer_b = TfidfVectorizer().fit(df_uni_br['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer_b.transform(df_uni_br['NOME_DA_IES']) # geral
tfidf_matrix_instituto = vectorizer_b.transform(df_uni_instituto['NOME_DA_IES'])
tfidf_matrix_faculdades = vectorizer_b.transform(df_uni_faculdades['NOME_DA_IES'])
tfidf_matrix_escola = vectorizer_b.transform(df_uni_escola['NOME_DA_IES'])
tfidf_matrix_universidade = vectorizer_b.transform(df_uni_universidade['NOME_DA_IES'])
tfidf_matrix_instituicao = vectorizer_b.transform(df_uni_instituicao['NOME_DA_IES'])
tfidf_matrix_outros = vectorizer_b.transform(df_uni_outros['NOME_DA_IES'])

# Vetorização TF-IDF : Lista IES Mundial
vectorizer_w = TfidfVectorizer().fit(df_uni_world['NOME_DA_IES'])
tfidf_matrix_world = vectorizer_w.transform(df_uni_world['NOME_DA_IES'])

In [ ]:
def iterate_sigla(nome_instituicao):

    #lista = df_uni["SIGLA"].apply(lambda x: True if (x + ' ' in nome_instituicao) or (' ' + x in nome_instituicao) or (' ' + x + ' ' in nome_instituicao) else False)
    lista = df_uni_sigla["SIGLA"].apply(lambda x: bool(re.search(rf'\b{x}\b', nome_instituicao)))
    siglas_encontradas = df_uni_sigla[lista]

    if siglas_encontradas.empty:
        return '', ''
    else:
        return siglas_encontradas.iloc[0]['SIGLA'], siglas_encontradas.iloc[0]['PAIS']

## UltimaFiliacaoEmIES

In [ ]:
threshold = 0.8

def metodo1(nomes_instituicoes):

    if nomes_instituicoes:

        nomes_instituicoes_split = str(nomes_instituicoes).split(';')

        for nome_instituicao in nomes_instituicoes_split:

            nome_instituicao = nome_instituicao.strip()

            #print(nome_instituicao)

            # Ver se contém a sigla
            sigla_detectado, _ = iterate_sigla(nome_instituicao)
            if sigla_detectado:
                #print("sigla detectado : ", sigla_detectado)
                #print("==================")
                return '1'

            # Se nao contem a sigla, procurar por nome completo
            else:
                vector_comparacao = vectorizer_b.transform([nome_instituicao])

                uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

                similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                max_value = np.max(similaridade)
                max_index = np.argmax(similaridade)

                nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                if (max_value >= threshold):
                    #print("ies br detectado : ", nome_detectado, max_value)
                    #print("==================")
                    return '1'

                # se nao achou nenhuma ies br, procurar ies estrangeira
                else:
                    vector_comparacao = vectorizer_w.transform([nome_instituicao])
                    uni_list_df, tfidf_matrix_df = df_uni_world, tfidf_matrix_world
                    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                    max_value = np.max(similaridade)
                    max_index = np.argmax(similaridade)

                    nome_detectado = uni_list_df.iloc[max_index]['NOME_DA_IES']

                    if (max_value >= threshold):
                        #print("ies estrangeiro detectado : ", nome_detectado, max_value)
                        #print("==================")
                        return '1'

                    else:
                        print("==================")

                    # nao eh ies

        print("==================")

    return '0'

df_doutores['UltimaFiliacaoEmIES'] = df_doutores["UltimaFiliacao"].apply(lambda x: metodo1(x))

In [ ]:
df_doutores['UltimaFiliacaoEmIES']

## HistoricoFiliacoesEmIESInternacional

In [ ]:
def metodo1(nomes_instituicoes):

    paises_detectados = []
    tem_historico = 0

    if nomes_instituicoes:

        nomes_instituicoes_split = str(nomes_instituicoes).split(';')

        for nome_instituicao in nomes_instituicoes_split:

            nome_instituicao = nome_instituicao.strip()

            #print(nome_instituicao)

            # Ver se contém a sigla
            sigla_detectado, pais = iterate_sigla(nome_instituicao)
            if pais:
                paises_detectados.append(pais)
                if pais != 'BR':
                    tem_historico = 1

            # Se nao contem a sigla, procurar por nome completo br
            else:
                vector_comparacao = vectorizer_b.transform([nome_instituicao])

                uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

                similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                max_value = np.max(similaridade)

                if (max_value >= threshold):
                    paises_detectados.append('BR')

                # se nao achou nenhuma ies br, procurar uma ies estrangeira
                else:
                    vector_comparacao = vectorizer_w.transform([nome_instituicao])
                    uni_list_df, tfidf_matrix_df = df_uni_world, tfidf_matrix_world
                    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

                    max_value = np.max(similaridade)

                    if (max_value >= threshold):
                        paises_detectados.append(pais)
                        tem_historico = 1

                    else:
                        print("==================")

                    # nao eh ies

        #print("==================")
    return tem_historico, paises_detectados

df_doutores['ListaHistoricoFiliacoesEmIESInternacional'] = ''
df_doutores['HistoricoFiliacoesEmIESInternacional'], df_doutores['ListaHistoricoFiliacoesEmIESInternacional'] = df_doutores["ListaFiliacoes"].apply(lambda x: metodo1(x))

In [ ]:
df_doutores['ListaHistoricoFiliacoesEmIESInternacional']

# Export


In [ ]:
df['PrimeiraArea'] = df['GrandesAreas'].str.split(';').str[0]
df['PrimeiraArea']

# Create a mapping dictionary
unique_areas = df['PrimeiraArea'].unique()
area_to_numeric = {area: idx for idx, area in enumerate(unique_areas)}

# Map the areas to numeric values
df['PrimeiraAreaNumeric'] = df['PrimeiraArea'].map(area_to_numeric)

print(area_to_numeric)

In [ ]:
df_doutores['ExclusividadeDoutorado'] = df_doutores['ExclusividadeDoutorado'].astype(float).astype(int)
df_doutores['Genero'] = df_doutores['Genero'].astype(float).astype(int)

df_doutores = df_doutores.astype({
    'IDLattes': int,
    'DuracaoDoutorado': float,
    'Genero': int,
    'GrandesAreas': str,
    'ExclusividadeDoutorado': int,
    'Publicacoes': float,
    'IdadeAcademica': str,
    'HistoricoFiliacoesEmIESInternacional': str,
    'UltimaFiliacaoEmIES': int,
    'ListaHistoricoFiliacoesEmIESInternacional': str
})

#    'ListaFiliacoes': str,
#    'UltimaFiliacao': str,

df_doutores.to_csv('doutores_processado_2000_zhong.csv')

In [ ]:
df_doutores['ExclusividadeDoutorado'].isna().count()

# Analise

In [ ]:
# df_doutores_processado = pd.read_csv('doutores_processado.csv', encoding='UTF-8')

## Corr

In [ ]:
import matplotlib.pyplot as plt

df_doutores_filtered = df_doutores[['DuracaoDoutorado', 'Publicacoes', 'IdadeAcademica', 'HistoricoFiliacoesEmIESInternacional', 'UltimaFiliacaoEmIES']]

rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = df_doutores_filtered.corr()
corr.style.background_gradient(cmap='coolwarm')

## Gower + Kmedoids

In [ ]:
import gower
import pandas as pd
from sklearn_extra.cluster import KMedoids
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

# Gower distance
gower_dist = gower.gower_matrix(df_doutores_filtered)

# K-Medoids
kmedoids = KMedoids(n_clusters=3, metric='precomputed', method='pam', random_state=0)
kmedoids.fit(gower_dist)
labels = kmedoids.labels_

# Plot
pca = PCA(n_components=2)
doutores_pca = pca.fit_transform(gower_dist)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(doutores_pca[:, 0], doutores_pca[:, 1], c=labels, cmap='viridis', s=50)
plt.title("Cluster Plot")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(scatter, label='Cluster Label')
plt.show()

## Logit

In [ ]:
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

def regressao_logistica(X, y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train.values.ravel())

    y_pred = model.predict(X_test)

    # resultado
    p,r,f,_none = precision_recall_fscore_support(y_test, y_pred, average='micro')
    print('P =',"{:2.2f}".format(p).replace(".",","),end='')
    print('  R =',"{:2.2f}".format(r).replace(".",","),end='')
    print('  F =',"{:2.2f}".format(f).replace(".",","))

In [ ]:
# DuracaoDoutorado

regressao_logistica(df_doutores_filtered[['IdadeAcademica', 'Publicacoes', 'HistoricoFiliacoesEmIESInternacional', 'UltimaFiliacaoEmIES']],
                    df_doutores_filtered[['DuracaoDoutorado']])

In [ ]:
# IdadeAcademica

regressao_logistica(df_doutores_filtered[['DuracaoDoutorado', 'Publicacoes', 'HistoricoFiliacoesEmIESInternacional', 'UltimaFiliacaoEmIES']],
                    df_doutores_filtered[['IdadeAcademica']])

In [ ]:
# Publicacoes

regressao_logistica(df_doutores_filtered[['DuracaoDoutorado', 'IdadeAcademica', 'HistoricoFiliacoesEmIESInternacional', 'UltimaFiliacaoEmIES']],
                    df_doutores_filtered[['Publicacoes']])